In [8]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM,Dropout,Dense,Activation
import datetime
from sklearn.model_selection import train_test_split


data = pd.read_csv("/Users/igyuseog/Desktop/data09/funda_train.csv")
submit = pd.read_csv("/Users/igyuseog/Desktop/data09/submission.csv")

In [9]:
model = Sequential()

# 첫번째
model.add(LSTM(30,
               return_sequences = True,
               input_shape = (30,1)))
# 두번째
model.add(LSTM(64,
              return_sequences = False))
# 아웃풋
model.add(Dense(1, activation = "linear"))

# 컴파일
model.compile(loss = "mse", optimizer = "rmsprop")

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 30, 30)            3840      
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                24320     
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 28,225
Trainable params: 28,225
Non-trainable params: 0
_________________________________________________________________


In [10]:
store = list(set(data["store_id"].values))
seq_len = 30 # 1 ~ 30을 보고 31을 맞추자.
seqence_length = seq_len + 1
amount = []
for now in store:
    train = data[data["store_id"] == now]["amount"].values
    result = []
    for index in range(len(train) - seqence_length):
        result.append(train[index: index + seqence_length]) #30개씩저장
    
    target = 0
    normal_data = []
    for window in result:
        target = window[0]
        normal_data.append((window / window[0]) -1)
    result = np.array(normal_data)
    
    #train과 test set을 나눠서 해보자.
    X = result[:,:-1]
    y = result[:,-1]

    #shuffle을 안해도 됌
    x_train, x_test, y_train, y_test = train_test_split(X,y, shuffle = False, test_size = 0.1)

    # shape을 3차원으로!
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1],1))
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
    
    # 학습진행
    model.fit(x_train, y_train,
             validation_data=(x_test, y_test),
             batch_size = 10,
             epochs = 5)
    
    # 맨마지막으로 3번 더 진행
    # 맨마지막을 가지고 3번을 더해보자.
    graph = []
    temp = x_test[-1].reshape(1,-1,1)
    cnt = 0

    while cnt < 3:
        answer = model.predict(temp)
        graph.append(float(answer.squeeze()))
        temp2 = temp.squeeze(axis = 0)
        temp = np.concatenate([temp2,answer], axis = 0)
        temp = temp.reshape(1,-1,1)
        cnt += 1
    
    predict = list(map(lambda x: (x+1)*target, graph))
    amount.append(sum(predict))

Epoch 1/5
401/401 [==============================] - 11s 20ms/step - loss: 19.1849 - val_loss: 9.9064
Epoch 2/5
401/401 [==============================] - 7s 18ms/step - loss: 14.5913 - val_loss: 9.4949
Epoch 3/5
401/401 [==============================] - 7s 18ms/step - loss: 9.9277 - val_loss: 8.8081
Epoch 4/5
401/401 [==============================] - 7s 18ms/step - loss: 11.8941 - val_loss: 8.8677
Epoch 5/5
401/401 [==============================] - 7s 18ms/step - loss: 12.8217 - val_loss: 9.1834
Epoch 1/5
338/338 [==============================] - 7s 19ms/step - loss: 0.4170 - val_loss: 0.5138
Epoch 2/5
338/338 [==============================] - 6s 18ms/step - loss: 0.3987 - val_loss: 0.4826
Epoch 3/5
338/338 [==============================] - 6s 18ms/step - loss: 0.3975 - val_loss: 0.4815
Epoch 4/5
338/338 [==============================] - 6s 18ms/step - loss: 0.3954 - val_loss: 0.4805
Epoch 5/5
338/338 [==============================] - 6s 17ms/step - loss: 0.3883 - val_loss: 0.

283/283 [==============================] - 5s 18ms/step - loss: 6.8476 - val_loss: 2.6990
Epoch 4/5
283/283 [==============================] - 5s 18ms/step - loss: 6.7931 - val_loss: 2.6922
Epoch 5/5
283/283 [==============================] - 5s 18ms/step - loss: 6.8003 - val_loss: 2.6757
Epoch 1/5
688/688 [==============================] - 12s 17ms/step - loss: 1.8214 - val_loss: 1.4067
Epoch 2/5
688/688 [==============================] - 12s 17ms/step - loss: 1.7899 - val_loss: 1.4187
Epoch 3/5
688/688 [==============================] - 12s 17ms/step - loss: 1.7779 - val_loss: 1.4448
Epoch 4/5
688/688 [==============================] - 12s 17ms/step - loss: 1.7883 - val_loss: 1.4213
Epoch 5/5
688/688 [==============================] - 11s 16ms/step - loss: 1.7734 - val_loss: 1.4306
Epoch 1/5
284/284 [==============================] - 5s 17ms/step - loss: 86.1848 - val_loss: 3.5078
Epoch 2/5
284/284 [==============================] - 5s 18ms/step - loss: 82.0074 - val_loss: 3.5748
Epo

432/432 [==============================] - 11s 26ms/step - loss: 37.6491 - val_loss: 24.0942
Epoch 4/5
432/432 [==============================] - 11s 26ms/step - loss: 37.1528 - val_loss: 23.1828
Epoch 5/5
432/432 [==============================] - 11s 26ms/step - loss: 37.3431 - val_loss: 22.8070
Epoch 1/5
258/258 [==============================] - 7s 28ms/step - loss: 2753.6523 - val_loss: 8.1725
Epoch 2/5
258/258 [==============================] - 6s 25ms/step - loss: 2733.8743 - val_loss: 8.2445
Epoch 3/5
258/258 [==============================] - 9s 34ms/step - loss: 2722.4316 - val_loss: 8.4363
Epoch 4/5
258/258 [==============================] - 8s 31ms/step - loss: 2712.5640 - val_loss: 9.1859
Epoch 5/5
258/258 [==============================] - 8s 30ms/step - loss: 2711.5642 - val_loss: 9.6783
Epoch 1/5
322/322 [==============================] - 10s 30ms/step - loss: 17990182.0000 - val_loss: 389.8661
Epoch 2/5
322/322 [==============================] - 10s 30ms/step - loss: 1

591/591 [==============================] - 13s 21ms/step - loss: 157.6746 - val_loss: 8.4886
Epoch 3/5
591/591 [==============================] - 13s 21ms/step - loss: 167.0096 - val_loss: 8.0872
Epoch 4/5
591/591 [==============================] - 13s 22ms/step - loss: 162.8766 - val_loss: 8.1526
Epoch 5/5
591/591 [==============================] - 13s 21ms/step - loss: 166.1265 - val_loss: 8.3512
Epoch 1/5
206/206 [==============================] - 4s 21ms/step - loss: 23.0767 - val_loss: 133.1359
Epoch 2/5
206/206 [==============================] - 4s 21ms/step - loss: 22.6481 - val_loss: 134.1384
Epoch 3/5
206/206 [==============================] - 5s 22ms/step - loss: 21.8580 - val_loss: 134.6824
Epoch 4/5
206/206 [==============================] - 4s 22ms/step - loss: 21.5200 - val_loss: 134.4729
Epoch 5/5
206/206 [==============================] - 5s 24ms/step - loss: 21.3302 - val_loss: 134.2674
Epoch 1/5
671/671 [==============================] - 15s 22ms/step - loss: 1156.741

Epoch 2/5
640/640 [==============================] - 11s 18ms/step - loss: 309106.3125 - val_loss: 4.5836
Epoch 3/5
640/640 [==============================] - 12s 19ms/step - loss: 308680.1562 - val_loss: 4.4159
Epoch 4/5
640/640 [==============================] - 12s 19ms/step - loss: 308576.4688 - val_loss: 4.5861
Epoch 5/5
640/640 [==============================] - 11s 17ms/step - loss: 308533.8125 - val_loss: 4.5823
Epoch 1/5
94/94 [==============================] - 1s 16ms/step - loss: 2.2481 - val_loss: 48.0096
Epoch 2/5
94/94 [==============================] - 1s 15ms/step - loss: 2.0686 - val_loss: 47.8166
Epoch 3/5
94/94 [==============================] - 2s 18ms/step - loss: 1.9047 - val_loss: 47.8276
Epoch 4/5
94/94 [==============================] - 2s 20ms/step - loss: 1.7601 - val_loss: 47.9228
Epoch 5/5
94/94 [==============================] - 2s 17ms/step - loss: 1.7145 - val_loss: 48.0703
Epoch 1/5
54/54 [==============================] - 1s 16ms/step - loss: 2757.4451

...

610/610 [==============================] - 10s 16ms/step - loss: 2.3761 - val_loss: 5.4205
Epoch 1/5
816/816 [==============================] - 13s 16ms/step - loss: 7.6355 - val_loss: 4.4282
Epoch 2/5
816/816 [==============================] - 13s 16ms/step - loss: 5.8649 - val_loss: 4.4487
Epoch 3/5
816/816 [==============================] - 13s 16ms/step - loss: 6.6804 - val_loss: 4.4842
Epoch 4/5
816/816 [==============================] - 13s 16ms/step - loss: 6.6242 - val_loss: 4.5919
Epoch 5/5
816/816 [==============================] - 14s 17ms/step - loss: 5.7233 - val_loss: 4.5003
Epoch 1/5
618/618 [==============================] - 10s 17ms/step - loss: 13.2667 - val_loss: 1.7269
Epoch 2/5
618/618 [==============================] - 11s 18ms/step - loss: 9.6376 - val_loss: 1.6516
Epoch 3/5
618/618 [==============================] - 11s 18ms/step - loss: 7.5351 - val_loss: 1.7737
Epoch 4/5
618/618 [==============================] - 11s 18ms/step - loss: 9.7144 - val_loss: 1.7202

67/67 [==============================] - 1s 18ms/step - loss: 12.7812 - val_loss: 18.0115
Epoch 1/5
544/544 [==============================] - 10s 18ms/step - loss: 3.4810 - val_loss: 6.2748
Epoch 2/5
544/544 [==============================] - 10s 18ms/step - loss: 3.3603 - val_loss: 6.3333
Epoch 3/5
544/544 [==============================] - 10s 18ms/step - loss: 3.5588 - val_loss: 6.2792
Epoch 4/5
544/544 [==============================] - 10s 18ms/step - loss: 3.3428 - val_loss: 6.4781
Epoch 5/5
544/544 [==============================] - 10s 18ms/step - loss: 3.3040 - val_loss: 6.2884
Epoch 1/5
67/67 [==============================] - 1s 19ms/step - loss: 286012.5625 - val_loss: 18.8405
Epoch 2/5
67/67 [==============================] - 1s 19ms/step - loss: 286850.3438 - val_loss: 1.9028
Epoch 3/5
67/67 [==============================] - 1s 19ms/step - loss: 287083.5312 - val_loss: 1.8407
Epoch 4/5
67/67 [==============================] - 1s 18ms/step - loss: 287983.4062 - val_loss:

Epoch 5/5
51/51 [==============================] - 1s 19ms/step - loss: 2571.7036 - val_loss: 142584.8281
Epoch 1/5
472/472 [==============================] - 7s 14ms/step - loss: 1051.8799 - val_loss: 14.3275
Epoch 2/5
472/472 [==============================] - 7s 15ms/step - loss: 892.7678 - val_loss: 11.0085
Epoch 3/5
472/472 [==============================] - 7s 14ms/step - loss: 1129.5438 - val_loss: 2.0917
Epoch 4/5
472/472 [==============================] - 7s 14ms/step - loss: 1096.1862 - val_loss: 1.9748
Epoch 5/5
472/472 [==============================] - 7s 14ms/step - loss: 964.5268 - val_loss: 11.1977
Epoch 1/5
97/97 [==============================] - 2s 20ms/step - loss: 377.7836 - val_loss: 5.0895
Epoch 2/5
97/97 [==============================] - 2s 19ms/step - loss: 378.9486 - val_loss: 5.4837
Epoch 3/5
97/97 [==============================] - 2s 19ms/step - loss: 412.4479 - val_loss: 6.3644
Epoch 4/5
97/97 [==============================] - 2s 19ms/step - loss: 362.26

60/60 [==============================] - 1s 21ms/step - loss: 897.3572 - val_loss: 7.1423
Epoch 1/5
443/443 [==============================] - 9s 21ms/step - loss: 118.4691 - val_loss: 23.8810
Epoch 2/5
443/443 [==============================] - 9s 21ms/step - loss: 128.7347 - val_loss: 28.2089
Epoch 3/5
443/443 [==============================] - 9s 21ms/step - loss: 134.5259 - val_loss: 28.9586
Epoch 4/5
443/443 [==============================] - 9s 21ms/step - loss: 89.4928 - val_loss: 25.2013
Epoch 5/5
443/443 [==============================] - 9s 21ms/step - loss: 132.5497 - val_loss: 19.9526
Epoch 1/5
728/728 [==============================] - 15s 21ms/step - loss: 6735.2290 - val_loss: 16253.3164
Epoch 2/5
728/728 [==============================] - 15s 21ms/step - loss: 6543.2583 - val_loss: 15484.2266
Epoch 3/5
728/728 [==============================] - 15s 21ms/step - loss: 6579.6064 - val_loss: 16296.1729
Epoch 4/5
728/728 [==============================] - 15s 21ms/step - los

KeyboardInterrupt: 

In [12]:
len(amount)

1188

In [17]:
submit["amount"][:1188] = amount

In [20]:
submit.to_csv("/Users/igyuseog/Desktop/open data/RNN.csv")